In [ ]:
from ultralytics import YOLO
import cv2

In [ ]:
model = YOLO(r"C:\Users\yhlin\ece342\project\ECE342-CV_Model\runs\detect\train4\weights\best.pt")

In [ ]:
img1 = cv2.imread("img2.jpg")
img2_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
cv2.imwrite("img2_gray.jpg", img2_gray)

In [ ]:
metrics = model.val(data=r"C:\Users\yhlin\ece342\project\ECE342-CV_Model\Face-Detection-1\data.yaml", split="test")

In [16]:
# Run batched inference on a list of images
results = model(["img2.jpg", "img2_gray.jpg"])  # return a list of Results objects

# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk


0: 640x480 1 face, 99.3ms
1: 640x480 1 face, 99.3ms
Speed: 8.2ms preprocess, 99.3ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
